In [6]:
import re
import pandas as pd
import argparse
from openpyxl import load_workbook
pd.options.display.max_rows = None

In [7]:
# 開啟檔案
def open_brd_file(filepath):
    f = open(filepath)
    return f

#用空格拆分每一行
def splitRow(string):
    string = string.split()
    return string

#刪除|開頭
def deleteRow1(string):
    pattern = re.compile(r"\|")
    delete = pattern.findall(string)
    return delete

#刪除- - -開頭
def deleteRow2(string):
    pattern = re.compile(r"^\-$")
    delete = pattern.findall(string)
    return delete

#刪除R開頭
def deleteRow3(string):
    pattern = re.compile(r"^R")
    delete = pattern.findall(string)
    return delete

#刪多餘的raw
def deleteNullVIAROW(df):
    for i in range(df.shape[0]):
        if df.loc[i, "location"] == "VIA" and df.loc[i, "gap"] == 0.0:
            df.drop(i, axis=0, inplace=True)

# 存excel
def save_excel(write, df, sheetName):
#     df = pd.DataFrame(datalist)
    df.to_excel(write, sheet_name=f'{sheetName}', index=False)

    
#計算表層裡層
def cauculateMSSL(pathIDX):
    TTL = 0
    MS = 0
    SL = 0
    NAN = 0
    
    #MS SL
    for idx in pathIDX:
        TTL += dfSQSR.loc[idx, "gap"]
        ####################################################  前面的表層裡層         
        if re.findall("BOTTOM|TOP", dfSQSR.loc[idx, "layer"]): 
            MS += dfSQSR.loc[idx, "gap"]
        elif re.findall("L\d+$|IN\d+$", dfSQSR.loc[idx, "layer"]):
            SL += dfSQSR.loc[idx, "gap"]
        else:
            NAN += dfSQSR.loc[idx, "gap"]
        
        ##########################################################
    return TTL, MS, SL, NAN

#這一份txt total的path 數量
def branchPathNum(df):
    #扣掉net name/start_end/total length
    maxPathNum = int((df.shape[1] - 7) / 2)
    return maxPathNum

#這一份txt total的path 數量
def branchPathNum(df):
    #扣掉net name/start_end/total length
    maxPathNum = int((df.shape[1] - 1) / 6)
    return maxPathNum

In [8]:
##########################RUN CODE######################################
#step1: 開啟檔案
filepath = "VRAM-0115a.txt"
try:
    f = open_brd_file(filepath)
    print(f"step1: Open file. (File name: {filepath}) -DONE")
except:
    print(f"File is not exit, check file name.")

#step2: 刪除不必要行數
rawData = []
SQS = []
summary = []
netNameList = []

for i in f:
    i = i.replace("\n","")#去掉換行符號
    row = splitRow(i)

    if not deleteRow1(row[0]):
        if not deleteRow2(row[0]):
            rawData.append(row)
print(f"step2: Clear excess rows. -DONE")

#step3: 整理成需要的格式
for i in rawData:
#     if len(i) == 1: #netname那一行

    if len(i) < 3: #netname那一行 ######################### TODO這行應該要改
        data = {} #清空data
        netName = i[0]
        
        netNameList.append(netName) #建立netnamelist
    elif i[-1] == "0.000" or i[-1] == "0.00": #找到location

        data = {} #清空 
        try:
            location, length = i[0], float(i[-1])
            data.update({"net_name" : netName, "location" : location, "length": length, "layer": ""})
        except:
            print(netName," has error.")
        
        SQS.append(data)
    elif i[-1] == "mils": #找到TOTAL
        data = {} #清空
        ttlLength = float(i[3])
#         data.update({"net_name" : netName, "total_length" : ttlLength})
        data.update({"net_name" : netName })
        summary.append(data)
        
    else:
        data = {} #清空
        if not deleteRow3(i[0]):
            try:
                location, length, layer = i[0], float(i[-2]), i[-1]
                data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
            except:
                print(netName, "has error.")

            SQS.append(data)

#存取df
dfsummary, dfSQS, dfSQSR = pd.DataFrame(summary), pd.DataFrame(SQS), pd.DataFrame(SQS)
print("step3: Transfer file format. -DONE")


#step4: 分析txt
#修改df
print("step4: Parsing txt file...")
print("       Calculating... - 25%")
#修改df
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    
    # 填第一個元件的layer空格 
    # ex. TOP = TOP
    dfSQSR.loc[indexList[0],"layer"] = dfSQSR.loc[indexList[1],"layer"]
    
    #每個netname裡面找gap
    length = len(indexList)
    for index in indexList[::-1]:
        if length > 1:
            gap = dfSQSR.loc[index]["length"] - dfSQSR.loc[index-1]["length"]
            dfSQSR.loc[index,"gap"] = gap
        else:
            gap = 0
            dfSQSR.loc[index,"gap"] = gap
        length -= 1

print("       Calculating... - 50%")
#刪除多餘的VIA
deleteNullVIAROW(dfSQSR)

File is not exit, check file name.


NameError: name 'f' is not defined